## Workflow to generate ensemble parameter combinations

In [ ]:
# this loads previoulsy stored environment
import dill
dill.load_session('opt_param_env.db')

In [4]:
# Load in necessary packages
import os
import pandas as pd
import numpy as np
import glob as glob
import datetime
from pyDOE import *

# personal functions
from funcs import unfold, tup_convert

### 1. Create dataframe with all parameters

* phenology
    - staygreen *
    - juv_leaves *
    - maxElongRate: max elongation rate at optimal temperature (12 cm/day)
    - rmax_lir: max leaf initation rate (leaves/day) *
    - rmax_ltar: max leaf tip appearance rate (leaves/day) *
* morphology
    - LAF: leaf angle factor for corn leaves (1.37)
    - f: spectral correction (0.15) *
    - scatt: leaf reflectance and transmittance (0.15) *
    - leaf_wid: leaf width (0.05 m) *
    - WLRARIO: leaf width to length ratio (0.106)
    - A_LW: leaf area coefficient with respect to LxW (0.75)
* physiology
    - vcm25: *
    - vpm25: *
    - g0: *
    - g1: *
    - t_opt: 
* water stress
    - psi_threshold: 

In [5]:
# genetic parameter list
params = ['staygreen', 'rmax_lir', 'rmax_ltar', 'juv_leaves', # phenology
          'leaf_wid', 'WLRATIO', 'A_LW', 'LAF', 'f', 'scatt', # morphology
          'vcm_25', 'vpm_25', 'g0', 'g1', 't_opt', # physiology
          'psi_threshold'] # water stress

#### ensemble paramter range

In [6]:
# 200228: range of these parameters still need to be checked/updated
param_range = {'staygreen':[1,6],
               'rmax_lir': [0.1, 0.5],
               'rmax_ltar': [0.1, 1.5],
               'juv_leaves': [12, 20],
               'leaf_wid': [10, 50],
               'WLRATIO': [1, 2],
               'A_LW': [0.5, 1],
               'LAF': [1, 3],
               'f': [0, 1],
               'scatt': [0, 1],
               'vcm_25': [40, 60],
               'vpm_25': [60, 80],
               'g0': [0.02, 0.1], # can check the range of these values from literature maybe?
               'g1': [2, 4],      # ball-berry slope & intercept
               't_opt': [20, 28],
               'psi_threshold': [-5, -1]}

### 2. Use Latin hypercube to find parameter combinations

info on this method here: https://pythonhosted.org/pyDOE/randomized.html

In [13]:
sample = lhs(len(params), samples=5) # trying with 5 cultivar combinations for now
df_params = pd.DataFrame(sample, columns=params)

for i in params: 
    df_params[i] = unfold(df_params[i], param_range[i][0], param_range[i][1])

# constraining leaf number to be integers only
df_params.juv_leaves = [round(x) for x in df_params.juv_leaves]
df_params.head()

,staygreen,rmax_lir,rmax_ltar,juv_leaves,leaf_wid,WLRATIO,A_LW,LAF,f,scatt,vcm_25,vpm_25,g0,g1,t_opt,psi_threshold
0,4.636344,0.167052,1.148629,19,44.628155,1.982931,0.781984,2.572780,0.583087,0.348595,40.770779,66.715961,0.058517,2.139435,22.824154,-3.784251
1,5.688691,0.445622,0.698849,13,39.460755,1.286827,0.688399,1.300634,0.391575,0.841593,48.486337,61.761481,0.037648,2.451947,26.753159,-1.409727
2,2.968741,0.197849,0.149925,14,20.776298,1.556972,0.568637,1.743317,0.629580,0.610220,52.592014,79.333786,0.078287,3.600594,20.563715,-2.275878
3,3.263489,0.377947,1.359031,18,32.878365,1.045214,0.937936,2.881544,0.003626,0.191044,46.409871,69.102897,0.095412,2.877680,24.128961,-4.338101
4,1.352633,0.320114,0.557453,16,12.286220,1.773894,0.833805,2.081080,0.994249,0.534909,57.741600,74.315708,0.025802,3.436221,24.868926,-3.091278


### 3. Setting up planting dates

In [14]:
# planting date range
season_start = datetime.datetime(1990,3,1)
season_end = datetime.datetime(1990,5,31)
dates = np.arange(season_start, season_end, datetime.timedelta(3))

pdates = []
for i in dates:
    pdates.append((str(i)[4:10]))

### 4. Creating cultivar init file template

#### constant parameters:

In [15]:
# cultivar traits
(gdd_mature, daylen_sen, LA_min, phyllo) = (1600, 0, 100, 3)

# soil water
(rrrm, rrry, rvrl) = (166.7, 31.3, 0.73)
(alpm, alpy, rtwl, rtminwtperunitarea) = (0.35, 0.04, 1.059e-004, 2.0000e-004)
(epsi, lupw, courmax) = (1, 1, 1)
(diffgeo1, diffgeo2, diffgoe3) = (2.4, 2.9, 0)
(wl, wa, wr, wb) = (1, 1, 2, 10)

# soil nitrogen
(isink, rroot) = (0, 0.017)
(consti_1, constk_1, cmin0_1) = (35.0, 0.5, 0.01)
(consti_2, constk_2, cmin0_2) = (17.2, 0.75, 0.30)

# photosynthesis
(eavp, eavc, eaj, hj, sj, jm_25, rd_25, ear) = (75100, 55900, 32800, 220000, 702.6, 300, 2, 39800)
(Kc_25, Ko_25, Kp_25, gbs, gi, gamma1) = (650, 450, 80, 0.003, 1, 0.193)

In [16]:
# setting up strings
def strings(): 
    tab = '    '
    str1 = '*** Corn growth simulation for US maize simualtion\n'
    str2 = 'cultivar: Romario\n'
    str3 = ('gdd_mature' + tab + 'juv_leaf' + tab + 'daylen_sen' + tab + 'staygreen' + tab + 
            'LA_min' + tab + 'Rmax_LTAR' + tab + 'Rmax_LTIR' + tab + 'phyllo\n')
    str4 = '\n'
    str5 = tup_convert(['%.0f' %gdd_mature, tab, '%.0f' %juv_leaves, tab, '%.0f' %daylen_sen, tab, 
                        '%.f' %staygreen, '%.f' %LA_min, tab, 
                        '%.f' %rmax_ltar, tab, '%.0f' %rmax_lir, tab, '%.0f\n' %phyllo])

    str6 = '[SoilRoot]\n'
    str7 = '*** water uptake parameter information ***\n'
    str8 = 'RRRM' + tab + 'RRRY' + tab + 'RVRL\n'
    str9 = tup_convert(['%.2f' %rrrm, tab, '%.2f' %rrry, tab, '%.2f\n' %rvrl])
    str10 = 'ALPM' + tab + 'ALPY' + tab + 'RTWL' + tab + 'RtMinWtPerUnitArea\n'
    str11 = tup_convert(['%.2f' %alpm, tab, '%.2f' %alpy, tab, '%.7f' %rtwl, tab, '%.4f\n' %rtminwtperunitarea])

    str12 = '[RootDiff]\n'
    str13 = '*** root mover parameter information\n'
    str14 = 'EPSI' + tab + 'lUpW' + tab + 'CourMax\n'
    str15 = tup_convert(['%.f' %epsi, tab, '%.f' %lupw, tab, '%.f\n' %courmax])
    str16 = 'Diffusivity and geotrophic velocity\n'
    str17 = tup_convert(['%.1f' %diffgeo1, tab, '%.1f' %diffgeo2, tab, '%.1f\n' %diffgoe3])

    str18 = '[SoilNitrogen]\n'
    str19 = '*** nitrogen root update parameter infromation\n'
    str20 = 'ISINK' + tab + 'Rroot\n'
    str21 = tup_convert(['%.2f' %isink, tab, '%.2f\n' %rroot])
    str22 = 'ConstI' + tab + 'Constk' + tab + 'Cmin0\n'
    str23 = tup_convert(['%.1f' %consti_1, tab, '%.1f' %constk_1, tab, '%.2f\n' %cmin0_1])
    str24 = tup_convert(['%.1f' %consti_2, tab, '%.2f' %constk_2, tab, '%.2f\n' %cmin0_2])

    str25 = '[Gas_Exchange Species Parameters]\n'
    str26 = '*** for photosynthesis calculations ***\n'
    str27 = ('EaVP' + tab + 'EaVc' + tab + 'Eaj' + tab + 'Hj' + tab + 'Sj' + tab + 'Vpm25' +
             tab + 'Vcm25' + tab + 'Jm25' + tab + 'Rd25' + tab + 'Ear' + tab + 'g0' + tab + 'g1\n')
    str28 = '\n'
    str29 = tup_convert(['%.f' %eavp, tab, '%.f' %eavc, tab, '%.f' %eaj, tab, '%.f' %hj, tab, '%.f' %sj, tab, 
                         '%.f' %vpm_25, tab, '%.f' %vcm_25, tab, '%.f' %jm_25, tab, '%.f' %rd_25, tab,
                         '%.f' %ear, tab, '%.f' %g0, tab, '%.f\n' %g1])

    str30 = '*** second set of parameters for photosynthesis ***\n'
    str31 = 'f' + tab + 'scatt' + tab + 'Kc_25' + tab + 'Ko_25' + tab + 'Kp_25' + tab + 'gbs' + tab + 'gi' + tab + 'gamma1\n'
    str32 = tup_convert(['%.2f' %f, tab, '%.2f' %scatt, tab, '%.2f' %Kc_25, tab, '%.2f' %Ko_25, tab, 
                         '%.2f' %Kp_25, tab, '%.2f' %gbs, tab, '%.2f' %gi, tab, '%.2f\n' %gamma1])

    # combining strings
    strs = [str1, str2, str3, str4, str5, str6, str7, str8, str9, str10, 
            str11, str12, str13, str14, str15, str16, str17, str18, str19, str20, 
            str21, str22, str23, str24, str25, str26, str27, str28, str29, str30, str31, str32]
    
    return(strs)

#### parameter perturbation:

In [17]:
for i in np.arange(df_params.shape[0]):
    [staygreen, rmax_lir, rmax_ltar, juv_leaves, leaf_wid, WLRATIO, 
     A_LW, LAF, f, scatt, vcm_25, vpm_25, g0, g1, t_opt, psi_threshold] = df_params.iloc[i,:]
    strs = strings()
    var = open('/home/disk/eos8/ach315/upscale/inits/var/var_' + str(i) + '.txt', 'w')
    var.writelines(strs)
    var.close()

In [12]:
dill.dump_session('opt_param_env.db')